# Fetching google maps APIs

In this notebook we illustrate how to fetch the elevation and distance matrix APIs from google maps.

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import googlemaps
gmaps = googlemaps.Client(
    key="YOUR_API_KEY", queries_per_second=1, retry_over_query_limit=False
)    

In [3]:
# load the station dataset as a pandas dataframe
station = pd.read_csv(
    "station.csv", 
    parse_dates=["install_date","modification_date","decommission_date"]
)
# add the pronto shop station to the dataframe
pronto_shop = dict(
    station_id="Pronto shop", name="Pronto shop", lat=47.6173156, long=-122.3414776
)
station = station.append(pronto_shop, ignore_index=True).rename(columns={"long":"lng"})
# get corresponding records (a list of dict)
station_records = station[["station_id","lat","lng"]].to_dict(orient="records")
station_records[:3]

[{'station_id': 'BT-01', 'lat': 47.618418, 'lng': -122.35096399999999},
 {'station_id': 'BT-03', 'lat': 47.615829, 'lng': -122.348564},
 {'station_id': 'BT-04', 'lat': 47.616094, 'lng': -122.341102}]

In [4]:
# the API is fetched only if elevations.json does not exist
try:
    elevations = json.load(open("elevations.json","r"))
except FileNotFoundError:
    print("Fetching the elevations API")
    elevations = gmaps.elevation(station_records)
    # saving the response in JSON
    json.dump(elevations, open("elevations.json", "w"))

In [5]:
# the API is fetched only if distances.json does not exist
try:
    distances = json.load(open("distances.json","r"))
except FileNotFoundError:
    print("Fetching the distance matrix API")
    distances = {}
    for origin in station_records:
        station_id = origin["station_id"]
        distances[station_id] = gmaps.distance_matrix(
            origins=origin, 
            destinations=station_records, 
            mode="bicycling"
        )
    # saving the response in JSON
    json.dump(distances, open("distances.json", "w"))

In [6]:
# parsing the json into a dataframe
station_ids = [record["station_id"] for record in station_records]
df_distances = pd.DataFrame()
for station_id, records in distances.items():
    parsed_records = [
        dict(
            distance=record["distance"]["value"],
            duration=record["duration"]["value"]
        ) 
        for record in records["rows"][0]["elements"]
    ]
    _df = pd.DataFrame(parsed_records)
    _df["from_station_id"] = station_id
    _df["to_station_id"] = station_ids
    df_distances = df_distances.append(_df, ignore_index=True)

# saving to csv
df_distances.to_csv("distances.csv", index=False)
df_distances.head()

,distance,duration,from_station_id,to_station_id
0,0,0,BT-01,BT-01
1,423,109,BT-01,BT-03
2,1068,370,BT-01,BT-04
3,868,261,BT-01,BT-05
4,2067,640,BT-01,CBD-03


In [7]:
# parsing the json into a dataframe
parsed_records = [
    dict(
        elevation=record["elevation"],
        lat=record["location"]["lat"],
        lng=record["location"]["lng"]
    )
    for record in elevations
]
df_elevations = pd.DataFrame(parsed_records)
df_elevations["station_id"] = station_ids

# sanity check
assert np.allclose(df_elevations.lat, station.lat)
assert np.allclose(df_elevations.lng, station.lng)

# saving to csv
df_elevations.to_csv("elevations.csv", index=False)
df_elevations.head()

,elevation,lat,lng,station_id
0,37.360447,47.61842,-122.35096,BT-01
1,33.815262,47.61583,-122.34856,BT-03
2,34.272057,47.61609,-122.34110,BT-04
3,44.281830,47.61311,-122.34421,BT-05
4,48.483391,47.61073,-122.33245,CBD-03
